In [2]:
import pymysql
import re
from string import capwords
import nltk
from nltk.corpus import words as nltk_words
from nltk.corpus import stopwords
from Levenshtein import distance as l_distance

ENGLISH_WORDS = set(nltk_words.words())
STOPWORDS = set(stopwords.words('english'))

LookupError: 
**********************************************************************
  Resource 'corpora/words' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - 'C:\\Users\\andre/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\andre\\Anaconda3\\nltk_data'
    - 'C:\\Users\\andre\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\andre\\AppData\\Roaming\\nltk_data'
**********************************************************************

In [53]:
STATE_ABBREVIATIONS = set(['al',
                          'ak',
                          'az', 
                          'ar',
                          'ca',
                          'co',
                          'ct',
                          'de',
                          'fl',
                          'ga',
                          'hi',
                          'id',
                          'il',
                          'in',
                          'ia',
                          'ks',
                          'ky',
                          'la',
                          'me',
                          'ma',
                          'mi',
                          'mn',
                          'ms',
                          'mo',
                          'mt',
                          'ne',
                          'nv',
                          'nh',
                          'nj',
                          'nm',
                          'ny',
                          'nc',
                          'nd',
                          'oh',
                          'ok',
                          'or',
                          'pa',
                          'ri',
                          'sc',
                          'sd',
                          'tn',
                          'tx',
                          'ut',
                          'vt',
                          'va',
                          'wa',
                          'wv',
                          'wi',
                          'wy'])

In [15]:
CONN_INFO = {'host': 'digitaldemocracydb.chzg5zpujwmo.us-west-2.rds.amazonaws.com',
             'port': 3306,
             'db': 'AndrewTest',
             # 'db': 'DDDB2015Dec',
             'user': 'awsDB',
             'passwd': 'digitaldemocracy789'}

In [25]:
org_df = None
with pymysql.connect(**CONN_INFO) as cursor:
        query = """SELECT oid, name
                   FROM Organizations"""
        cursor.execute(query)

        org_lists = [[oid, name] for oid, name in cursor]
        org_df = pd.DataFrame(org_lists, columns=['oid', 'name'])


In [148]:
CITY_ABBREVIATIONS = set(['ny', 'la', 'sf', 'sd'])
ENDINGS = set(['lp', 'llc', 'inc', 'co', '&', 'limited', 'the', 'llp'])
STARTS = set(['the'])

def norm_name(name):
    name = unidecode(name).lower().strip()
    
    name = re.sub(r'\(*.\)', '', name)
    
    if ';' in name:
        split = name.split(';')
        name = split[1] + ' ' + split[0]
        
    match = re.match(r'(.+)(, city of)', name)
    if match:
        name = 'city of ' + match.group(1)
    match = re.match(r'(.+)(, county of)', name)
    if match:
        name = 'city of ' + match.group(1)
    
    words = [word.replace("'", '') for word in name.split(' ') if word.strip() != '']
    
    while re.sub('[,\.]', '', words[-1]) in ENDINGS:
        words = words[:-1]

    words[-1] = re.sub('[,\.]', '', words[-1])    
    if words[0] in STARTS:
        words = words[1:]
    
#     new_words = []
#     for word in words:
#         if word in CITY_ABBREVIATIONS:
#             new_words.append(word.upper())
#         elif word in STATE_ABBREVIATIONS:
#             new_words.append(word.upper())
#         elif len(word) <= 3 and word not in ENGLISH_WORDS:
#             new_words.append(word.upper())
#         else:
#             new_words.append(capwords(word))
#     words = new_words
              
    return ' '.join(words)

In [146]:
def hard_coded_matches(name):
    out = None
    if name == 'cta':
        out = "california teachers association"
    return out

In [149]:
org_df['normed_name'] = org_df['name'].apply(norm_name)
org_df['hard_coded_name'] = org_df['normed_name'].apply(hard_coded_matches)

In [177]:
# org_df.sample(20)

In [178]:
# org_df[org_df.name.apply(lambda x: 'California Teacher' in x)]

In [201]:
def match_other_rows(row, cpy_org_df):
    out_set = set()
    matched_rows = cpy_org_df[(cpy_org_df['normed_name'] == row['normed_name']) & 
                              (org_df['oid'] > row['oid'])
                              ]
    out_set.update(list(matched_rows.index))
    
    matched_rows = cpy_org_df[(cpy_org_df['hard_coded_name'] == row['normed_name']) & 
                              (org_df['oid'] > row['oid'])
                              ]
    out_set.update(list(matched_rows.index))
    return out_set

In [184]:
cpy_org_df = org_df.copy()

In [208]:
org_df['matched_indices'] = org_df.apply(lambda row: match_other_rows(row, cpy_org_df), axis=1)

In [216]:
org_df.sample(3)

,oid,name,normed_name,hard_coded_name,my_set,matched_indices
351,352,San Diego Unified Port District,san diego unified port district,None,"{1, 2}",{12854}
11096,16670,"Supervalu, Inc., A Delaware Corporation","supervalu, inc., a delaware corporation",None,"{1, 2}",{}
5435,5445,Triton Management Services,triton management services,None,"{1, 2}",{}


In [223]:
def mark_rows_for_deletion(row, cpy_org_df):
    matched_rows = cpy_org_df[cpy_org_df['matched_indices'].apply(lambda indices: row.name in indices)]
    if len(matched_rows.index) > 0:
        return True
    else:
        return False

In [221]:
cpy_org_df = org_df.copy()

In [225]:
org_df['delete'] = org_df.apply(lambda row: mark_rows_for_deletion(row, cpy_org_df), axis=1)

In [227]:
sum(org_df.delete)

8115

In [229]:
sum(~org_df.delete)

14287

In [230]:
org_df = org_df[~org_df['delete']]

In [ ]:
def update_dddb(cursor, data):
    update_stmt = """UPDATE %(table)s
                     SET %(field_name)s = %(new_oid)s
                     WHERE %(filed_name)s = %(old_oid)s"""
    cursor.execute(update_stmt, data)
    

In [ ]:
def update_oids(cursor, row):
    table_field_pairs = [('Organizations', 'oid'),
                         ('Gift', 'oid'),
                         ('Contribution', 'oid'),
                         ('LobbyistEmployer')]
    pass

In [232]:
org_df

,oid,name,normed_name,hard_coded_name,my_set,matched_indices,delete
0,1,The Messersmith Group,messersmith group,None,"{1, 2}","{2, 4, 12493}",False
1,2,Milberg Weiss Bershad Hynes & Lerach,milberg weiss bershad hynes & lerach,None,"{1, 2}",{12494},False
3,4,International Brotherhood Of Electrical Worker...,international brotherhood of electrical worker...,None,"{1, 2}",{12495},False
5,6,Sankyo Pharma,sankyo pharma,None,"{1, 2}",{12496},False
6,7,American College Of Obstetricians And Gynecolo...,american college of obstetricians and gynecolo...,None,"{1, 2}",{12497},False
7,8,California State Association Of Counties,california state association of counties,None,"{1, 2}",{20089},False
8,9,Associated General Contractors Of California,associated general contractors of california,None,"{1, 2}",{12498},False
9,10,California Association Of Harbor Masters And P...,california association of harbor masters and p...,None,"{1, 2}","{16979, 12499, 14963}",False
10,11,California Marine Affairs And Navigation Confe...,california marine affairs and navigation confe...,None,"{1, 2}",{12500},False
11,12,Central California Chamber Alliance,central california chamber alliance,None,"{1, 2}",{12501},False


In [132]:
%%timeit
d = org_df['name'].apply(lambda name: l_distance(name, 'Sierra Club'))

10 loops, best of 3: 43.7 ms per loop
